In [4]:
!pip install requests beautifulsoup4 lxml

     |████████████████████████████████| 3.2 MB 5.1 MB/s eta 0:00:01
Using legacy 'setup.py install' for lxml, since package 'wheel' is not installed.
    Running setup.py install for lxml ... done
You should consider upgrading via the '/Users/inoccu/.anyenv/envs/pyenv/versions/3.10.2/bin/python3.10 -m pip install --upgrade pip' command.


In [1]:
import requests

session = requests.session()
res = session.get('http://127.0.0.1:8000/calc/')

In [2]:
import bs4

soup = bs4.BeautifulSoup(res.text, 'lxml')
soup

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/static/calc/css/bootstrap.min.css" rel="stylesheet"/>
<title>商品検索</title>
</head>
<body>
<nav class="navbar navbar-expand-lg navbar-dark bg-primary">
<div class="container">
<a class="navbar-brand" href="/calc/">Calc</a>
<ul class="navbar-nav">
<li class="nav-item">
<a class="nav-link" href="/calc/">売上</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/items/">商品管理</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/orders/">売上一覧</a>
</li>
</ul>
</div>
</nav>
<div class="container" style="margin-top: 1em;">
<div class="container">
<h1>商品検索</h1>
<ul class="nav justify-content-center" style="margin: 1em auto;">
<li class="nav-item">
<a class="nav-link" href="/calc/">商品検索</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/cart/">ショッピングカート</a>
</li>
</

In [3]:
soup.find('title').text

'商品検索'

In [5]:
brand = soup.find('a', class_='navbar-brand')
print(brand)
print(brand['class'])
print(brand['href'])
print(brand.text)

<a class="navbar-brand" href="/calc/">Calc</a>
['navbar-brand']
/calc/
Calc


In [6]:
navbar_nav = soup.find('ul', class_='navbar-nav')
for a in navbar_nav.find_all('a', class_='nav-link'):
    print(a)
    print('class:', a['class'])
    print('href:', a['href'])
    print('text:', a.text)

<a class="nav-link" href="/calc/">売上</a>
class: ['nav-link']
href: /calc/
text: 売上
<a class="nav-link" href="/calc/items/">商品管理</a>
class: ['nav-link']
href: /calc/items/
text: 商品管理
<a class="nav-link" href="/calc/orders/">売上一覧</a>
class: ['nav-link']
href: /calc/orders/
text: 売上一覧


In [7]:
token = soup.find('input', attrs={'name': 'csrfmiddlewaretoken', 'type': 'hidden'})
token['value']

'h12nw7LciObIXROHjvIktsItIQ7XG2E3E0p0ejR3zQgd6xQZHFQWS6QDHDJnLtz2'

In [8]:
data = {
    'csrfmiddlewaretoken': token['value'],
    'code': '',
    'name': '',
    'price_min': '0',
    'price_max': '',
    'condition': 'and'
}
res = session.post('http://127.0.0.1:8000/calc/', data=data)
res

<Response [200]>

In [9]:
result = bs4.BeautifulSoup(res.text, 'lxml')
result

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/static/calc/css/bootstrap.min.css" rel="stylesheet"/>
<title>商品検索</title>
</head>
<body>
<nav class="navbar navbar-expand-lg navbar-dark bg-primary">
<div class="container">
<a class="navbar-brand" href="/calc/">Calc</a>
<ul class="navbar-nav">
<li class="nav-item">
<a class="nav-link" href="/calc/">売上</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/items/">商品管理</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/orders/">売上一覧</a>
</li>
</ul>
</div>
</nav>
<div class="container" style="margin-top: 1em;">
<div class="container">
<h1>商品検索</h1>
<ul class="nav justify-content-center" style="margin: 1em auto;">
<li class="nav-item">
<a class="nav-link" href="/calc/">商品検索</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/cart/">ショッピングカート</a>
</li>
</

In [10]:
items = []
table = result.find('table')
tbody = table.find('tbody')
for tr in tbody.find_all('tr'):
    td_list = tr.find_all('td')
    url = td_list[0].find('a')['href']
    code = td_list[0].find('a').text
    name = td_list[1].text
    price = int(td_list[2].text.replace(',', ''))
    items.append([code, url, name, price])
items

[['A01', '/calc/line/1', 'コーヒー', 200],
 ['A02', '/calc/line/2', 'カフェオレ', 400],
 ['B01', '/calc/line/3', 'マウス', 1000]]

In [11]:
import csv

with open('items.csv', 'w', newline='', encoding='UTF-8') as f:
    fieldnames = ['code', 'url', 'name', 'price']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    writer.writerows(items)

In [12]:
items = []

with open('items.csv', 'r', encoding='UTF-8') as f:
    reader = csv.reader(f)
    first = True
    for line in reader:
        if first:
            first = False
            continue
        items.append(line)
        
items

[['A01', '/calc/line/1', 'コーヒー', '200'],
 ['A02', '/calc/line/2', 'カフェオレ', '400'],
 ['B01', '/calc/line/3', 'マウス', '1000']]

In [13]:
new_items = []
for item in items:
    res = requests.get(f'http://127.0.0.1:8000{item[1]}')
    page = bs4.BeautifulSoup(res.text, 'lxml')
    img = page.find('img')
    if img is not None:
        item.append(img['src'])
    else:
        item.append('')
    new_items.append(item)

new_items

[['A01', '/calc/line/1', 'コーヒー', '200', ''],
 ['A02', '/calc/line/2', 'カフェオレ', '400', ''],
 ['B01', '/calc/line/3', 'マウス', '1000', '']]

In [14]:
with open('items2.csv', 'w', newline='', encoding='UTF-8') as f:
    fieldnames = ['code', 'url', 'name', 'price', 'image']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    writer.writerows(new_items)

# 課題（1）

In [15]:
session = requests.session()
res = session.get('http://127.0.0.1:8000/accounts/login/')
login = bs4.BeautifulSoup(res.text, 'lxml')
login

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/static/calc/css/bootstrap.min.css" rel="stylesheet"/>
<title></title>
</head>
<body>
<nav class="navbar navbar-expand-lg navbar-dark bg-primary">
<div class="container">
<a class="navbar-brand" href="/calc/">Calc</a>
</div>
</nav>
<div class="container" style="margin-top: 1em;">
<form action="/accounts/login/" method="POST">
<input name="csrfmiddlewaretoken" type="hidden" value="Y70wVR2lQYwXXnwNVLSsHUvVfuqrFygYyOHvJMKLNWu4aAhdKCqlDqkVPAU1wl2N"/>
<p>
<label for="id_username">ユーザー名:</label>
<input id="id_username" name="username" required="" type="text"/>
</p>
<p>
<label for="id_password">パスワード:</label>
<input id="id_password" name="password" required="" type="password"/>
</p>
<input type="submit" value="ログイン"/>
</form>
</div>
<script src="/static/calc/js/bootstrap.bundle.min.js"></scrip

In [16]:
token = login.find('input', attrs={'name': 'csrfmiddlewaretoken', 'type': 'hidden'})
token['value']

'Y70wVR2lQYwXXnwNVLSsHUvVfuqrFygYyOHvJMKLNWu4aAhdKCqlDqkVPAU1wl2N'

In [17]:
res = session.post('http://127.0.0.1:8000/accounts/login/', data={'csrfmiddlewaretoken': token['value'], 'username': 'admin', 'password': 'mm10hDmK'})
items_page = bs4.BeautifulSoup(res.text, 'lxml')

In [18]:
items_page

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/static/calc/css/bootstrap.min.css" rel="stylesheet"/>
<title>商品一覧</title>
</head>
<body>
<nav class="navbar navbar-expand-lg navbar-dark bg-primary">
<div class="container">
<a class="navbar-brand" href="/calc/">Calc</a>
<ul class="navbar-nav">
<li class="nav-item">
<a class="nav-link" href="/calc/">売上</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/items/">商品管理</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/orders/">売上一覧</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/accounts/logout/">ログアウト</a>
</li>
</ul>
</div>
</nav>
<div class="container" style="margin-top: 1em;">
<div class="container">
<h1>商品一覧</h1>
<ul class="nav">
<li class="nav-item"><a class="nav-link" href="/calc/add_item/">新規作成</a></li>
</ul>
<table class="table">
<thead>
<tr>
<

In [19]:
items = []
table = items_page.find('table')
tbody = table.find('tbody')
for tr in tbody.find_all('tr'):
    td_list = tr.find_all('td')
    url = td_list[0].find('a')['href']
    code = td_list[0].find('a').text
    name = td_list[1].text
    price = int(td_list[2].text.replace(',', ''))
    items.append([code, url, name, price])
items

[['A01', '/calc/item/1', 'コーヒー', 200],
 ['A02', '/calc/item/2', 'カフェオレ', 400],
 ['B01', '/calc/item/3', 'マウス', 1000]]

In [20]:
new_items = []
for item in items:
    res = session.get(f'http://127.0.0.1:8000{item[1]}')
    page = bs4.BeautifulSoup(res.text, 'lxml')
    img = page.find('img')
    if img is not None:
        item.append(img['src'])
    else:
        item.append('')
    new_items.append(item)
new_items

[['A01', '/calc/item/1', 'コーヒー', 200, '/media/images/coffee_i1kN7eU.jpg'],
 ['A02', '/calc/item/2', 'カフェオレ', 400, ''],
 ['B01', '/calc/item/3', 'マウス', 1000, '']]

In [21]:
with open('items2.csv', 'w', newline='', encoding='UTF-8') as f:
    fieldnames = ['code', 'url', 'name', 'price', 'image']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    writer.writerows(new_items)

# 課題（2）

In [22]:
session = requests.session()
res = session.get('http://127.0.0.1:8000/accounts/login/')
login = bs4.BeautifulSoup(res.text, 'lxml')

token = login.find('input', attrs={'name': 'csrfmiddlewaretoken', 'type': 'hidden'})
session.post('http://127.0.0.1:8000/accounts/login/', data={'csrfmiddlewaretoken': token['value'], 'username': 'admin', 'password': 'mm10hDmK'})

res = session.get('http://127.0.0.1:8000/calc/orders/')
orders_page = bs4.BeautifulSoup(res.text, 'lxml')
orders_page

<!DOCTYPE html>
<html lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/static/calc/css/bootstrap.min.css" rel="stylesheet"/>
<title>売上一覧</title>
</head>
<body>
<nav class="navbar navbar-expand-lg navbar-dark bg-primary">
<div class="container">
<a class="navbar-brand" href="/calc/">Calc</a>
<ul class="navbar-nav">
<li class="nav-item">
<a class="nav-link" href="/calc/">売上</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/items/">商品管理</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/calc/orders/">売上一覧</a>
</li>
<li class="nav-item">
<a class="nav-link" href="/accounts/logout/">ログアウト</a>
</li>
</ul>
</div>
</nav>
<div class="container" style="margin-top: 1em;">
<div class="container">
<h1>売上一覧</h1>
<table class="table">
<thead>
<tr>
<th>ID</th>
<th>売上日時</th>
<th>売上金額</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td><a href="/calc/orde

In [23]:
orders = []
table = orders_page.find('table')
tbody = table.find('tbody')
for tr in tbody.find_all('tr'):
    td_list = tr.find_all('td')
    order_id = td_list[0].text
    url = td_list[1].find('a')['href']
    order_datetime = td_list[1].text
    total_sum = int(td_list[2].text.replace(',', ''))
    order_lines = []
    
    res2 = session.get(f'http://127.0.0.1:8000{url}')
    order_page = bs4.BeautifulSoup(res2.text, 'lxml')
    lines = order_page.find_all('table', class_='table')[1]
    lines_tbody = lines.find('tbody')
    for line_tr in lines_tbody.find_all('tr'):
        line_td_list = line_tr.find_all('td')
        code = line_td_list[0].text
        name = line_td_list[1].text
        price = int(line_td_list[2].text.replace(',', ''))
        count = line_td_list[3].text
        sum = line_td_list[4].text
        order_lines.append({
            'item_code': code,
            'item_name': name,
            'price': price,
            'count': count,
            'sum': sum,
        })
    
    orders.append({
        'id': order_id,
        'order_datetime': order_datetime,
        'total_sum': total_sum,
        'order_lines': order_lines,
    })
    
orders

[{'id': '1',
  'order_datetime': '2022年2月28日20:26',
  'total_sum': 2400,
  'order_lines': [{'item_code': 'A01',
    'item_name': 'コーヒー',
    'price': 200,
    'count': '2',
    'sum': '400'},
   {'item_code': 'B01',
    'item_name': 'マウス',
    'price': 1000,
    'count': '2',
    'sum': '2,000'}]}]

In [24]:
import json

with open('orders.json', 'w', encoding='UTF-8') as f:
    json.dump(orders, f, ensure_ascii=False, indent=4)